In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import numpy as np
import os
import sys
import pickle
import seaborn as sns
sns.set_theme(style="whitegrid")
import warnings
warnings.filterwarnings("ignore")

# 1. Simulation running or results extraction

In [ ]:
#Run this for simulation

weather_year=2018 #must be between 2017 and 2019
carbon_tax=60
gas_price_coef=1
coal_price_coef=1 
number_of_sub_techs=5
fr_flex_consum={'conso':{"nbVE":5,"steel_twh":5,"H2_twh":25},'ratio':{"VE_ratio":0,"steel_ratio":0,"H2_ratio":0}}

if os.path.basename(os.getcwd()) == "Seven_node_Europe":
    os.chdir('../..')
    
from Main_2030 import *
Variables=main_2030(weather_year,carbon_tax,gas_price_coef,coal_price_coef,number_of_sub_techs,fr_flex_consum)

In [ ]:
#Run this to load a result file
with open("Result_2030.pickle", 'rb') as f:
    Variables=pickle.load(f)

# 2. Preliminary calculations

In [ ]:
states=["2030"]
exch_list=[Variables["exchange"]]
data=[Variables["energy"]]
data_stor=[Variables["storageOut"]]
data_capa=[Variables["capacity"]]
data_stor_capa=[Variables["Pmax"]]
consumption=Variables['total_consumption']
consumption["Date"]=pd.to_datetime(consumption["Date"])
consumption.sort_values(by='Date', inplace=True)
consumption.reset_index(inplace=True,drop=True)
colors=["black"]

In [ ]:
#subtechs list
l=data_capa[0]
l=l.TECHNOLOGIES.unique()
u=[]
n_list=[]
for tech_name in l:
    if tech_name[:4]=="Coal" and tech_name!="Coal":
        n_list.append(tech_name[4:])

# 3. Results plot

## a. Capacity

In [ ]:
# total_consum=round(consum.areaConsumption.sum()/10**6,2)
tech_list=["OldNuke","NewNuke","Coal","Lignite","CCG","TAC","HydroRiver","HydroReservoir",'HydroStorage',"Biomass",
           "WindOnShore","WindOffShore","Solar","Battery"]



df_capa=pd.DataFrame(columns=["TECHNOLOGIES"]+states)
df_capa["TECHNOLOGIES"]=np.array(tech_list)
# print(len(data))
for i in range(len(data)):
    capa=data_capa[i]
    for t in tech_list:
        t_list=[t]
        if len(n_list)>0:
            for j in range(len(n_list)):
                t_list.append(t+n_list[j])

        if t not in ["HydroStorage","Battery"] :
            val=round(capa[capa.TECHNOLOGIES.isin(t_list)].capacity.to_numpy().sum()/10**3,2)
    #         print(val)
            df_capa.loc[df_capa.TECHNOLOGIES==t,states[i]]=val
        else:
            u=data_stor_capa[i]
            val=round(u[u.STOCK_TECHNO==t].Pmax.to_numpy().sum()/10**3,2)
#             print(val)
            df_capa.loc[df_capa.TECHNOLOGIES==t,states[i]]=val
total_capa=df_capa[states[0]].sum()

In [ ]:
df_capa

In [ ]:
tech_list=["OldNuke","NewNuke","Coal","Lignite","CCG","TAC","HydroRiver","HydroReservoir",'HydroStorage',"Biomass",
           "WindOnShore","WindOffShore","Solar","Battery"]

for area in ["FR","DE","GB","BE","IT","ES","CH"]:
    print("\nCapacities installed for "+area)
    df_capa=pd.DataFrame(columns=["TECHNOLOGIES"]+states)
    df_capa["TECHNOLOGIES"]=np.array(tech_list)
    # print(len(data))
    for i in range(len(data)):
        capa=data_capa[i]
        capa=capa[capa.AREAS==area]
        for t in tech_list:
            t_list=[t]
            if len(n_list)>0:
                for j in range(len(n_list)):
                    t_list.append(t+n_list[j])

            if t not in ["HydroStorage","Battery"] :
                val=round(capa[capa.TECHNOLOGIES.isin(t_list)].capacity.to_numpy().sum()/10**3,2)
        #         print(val)
                df_capa.loc[df_capa.TECHNOLOGIES==t,states[i]]=val
            else:
                u=data_stor_capa[i]
                u=u[u.AREAS==area]
                val=round(u[u.STOCK_TECHNO==t].Pmax.to_numpy().sum()/10**3,2)
    #             print(val)
                df_capa.loc[df_capa.TECHNOLOGIES==t,states[i]]=val
    print(df_capa)

## b. Production

In [ ]:
country="FR"
p=Variables["capacity"]
p=p[p.AREAS==country]
p

### i. Production at the 7 nodes scale 

In [ ]:
# total_consum=round(consum.areaConsumption.sum()/10**6,2)
tech_list=["OldNuke","NewNuke","Coal","Lignite","CCG","TAC","HydroRiver","HydroReservoir",'HydroStorage',"Biomass",
           "WindOnShore","WindOffShore","Solar","Battery","curtailment"]



df_prod=pd.DataFrame(columns=["TECHNOLOGIES"]+states)
df_prod["TECHNOLOGIES"]=np.array(tech_list)

for i in range(len(data)):
    prod=data[i]
    for t in tech_list:
        t_list=[t]
        if len(n_list)>0:
            for j in range(len(n_list)):
                t_list.append(t+n_list[j])

        if t!="HydroStorage":
            val=round(prod[prod.TECHNOLOGIES.isin(t_list)].energy.to_numpy().sum()/10**6,2)
    #         print(val)
            df_prod.loc[df_prod.TECHNOLOGIES==t,states[i]]=val
        else:
            u=data_stor[i]
            val=round(u[u.STOCK_TECHNO==t].storageOut.to_numpy().sum()/10**6,2)

            df_prod.loc[df_prod.TECHNOLOGIES==t,states[i]]=val
total_prod=df_prod[states[0]].sum()

In [ ]:
df_prod_bar=pd.DataFrame(columns=["TECHNOLOGIES","Production (TWh)","Production (%)","Scenario"])
for c in df_prod.columns[1:]:
    u=df_prod[["TECHNOLOGIES",c]]
    u.rename(columns={c:"Production (TWh)"},inplace=True)
    u["Production (%)"]=u["Production (TWh)"]/u["Production (TWh)"].sum()*100
    u["Scenario"]=c
    df_prod_bar=pd.concat([df_prod_bar,u],ignore_index=True)

In [ ]:
import plotly.express as px
fig = px.bar(df_prod_bar, x='Scenario', y='Production (%)', color="TECHNOLOGIES")
fig.show()

In [ ]:
fig = px.pie(df_prod_bar, values='Production (TWh)', names='TECHNOLOGIES', title='Production per technology (in %)')
fig.show()

### ii. Production at country scale

In [ ]:
# total_consum=round(consum.areaConsumption.sum()/10**6,2)
countries=["FR","DE","BE","GB","IT","CH","ES"]
tech_list=["OldNuke","NewNuke","Biomass","Coal","Lignite","CCG","TAC","HydroRiver","HydroReservoir",'HydroStorage','Battery',
            "WindOnShore","WindOffShore","Solar"]



df_prod_country=pd.DataFrame(columns=["AREAS","TECHNOLOGIES","State","Production (TWh)"])


for i in range(len(states)):
    prod=data[i]
    for c in countries:
        for t in tech_list:
            t_list=[t]
            if len(n_list)>0:
                for j in range(len(n_list)):
                    t_list.append(t+n_list[j])
            if t not in ["HydroStorage","Battery"]:
                val=round(prod[(prod.TECHNOLOGIES.isin(t_list)) & (prod.AREAS==c)].energy.to_numpy().sum()/10**6,2)
                array=np.array([[c,t,states[i],val]])
                df_prod_country=pd.concat([df_prod_country,
                                           pd.DataFrame(array,columns=["AREAS","TECHNOLOGIES","State","Production (TWh)"])])
            else:
                u=data_stor[i]
                val=round(u[(u.STOCK_TECHNO==t) & (u.AREAS==c)].storageOut.to_numpy().sum()/10**6,2)
                array=np.array([[c,t,states[i],val]])
                df_prod_country=pd.concat([df_prod_country,
                                           pd.DataFrame(array,columns=["AREAS","TECHNOLOGIES","State","Production (TWh)"])])

df_prod_country["Production (TWh)"]=df_prod_country["Production (TWh)"].astype('float')
                
# comparison_per_country=df_prod_country.copy()
# for country in countries:
#     total_prod=df_prod[states[0]].sum()
#     total_consum=round(consum[consum.AREAS==country].areaConsumption.sum()/10**6,2)
#     comparison_per_country.loc[comparison_per_country.AREAS==country,"Production (TWh)"]= comparison_per_country.loc[comparison_per_country.AREAS==country,"Production (TWh)"].astype(float)/total_consum
    
# comparison_per_country["Difference with historical"]=np.zeros(len(comparison_per_country))
# for state in states:
#     comparison_per_country.loc[comparison_per_country.State==state,"Difference with "+states[0]]=(comparison_per_country.loc[comparison_per_country.State==state,"Production (TWh)"]-comparison_per_country.loc[comparison_per_country.State=="Historical","Production (TWh)"])*100                

In [ ]:
from  matplotlib.ticker import PercentFormatter
g = sns.catplot(x="State", y="Production (TWh)",hue="AREAS",col="TECHNOLOGIES",col_wrap=3,kind="bar",
                data=df_prod_country)
# for ax in g.axes.flat:
#     ax.yaxis.set_major_formatter(PercentFormatter())

plt.show()

In [ ]:
country="CH"
p=Variables["energy"]
p=p[p.AREAS==country]
storout=Variables["storageOut"]
#print("HydroStorage",round(storout[storout.AREAS==country].storageOut.sum()/10**6,2),"TWh")
storout=storout[(storout.AREAS==country) & (storout.STOCK_TECHNO!="Battery")]
storout=storout.rename(columns={'STOCK_TECHNO':'TECHNOLOGIES',"storageOut":"energy"})
p=pd.concat([p,storout],ignore_index=True)
tot=0
for t in p.TECHNOLOGIES.unique():
    l=p[p.TECHNOLOGIES==t]
    print(t,round(l.energy.sum()/10**6,2),"TWh")
    tot+=l.energy.sum()
print("Total produced",round(tot/10**6,2),"TWh")

In [ ]:
tech_list=["OldNuke","NewNuke","Biomass","Coal","CCG","HydroRiver","HydroReservoir",'HydroStorage',
           "WindOnShore","WindOffShore","Solar","curtailment"]
sorted_df=pd.DataFrame(columns=["Date","TECHNOLOGIES","energy"])
for i in range(len(tech_list)):
    d=p[p.TECHNOLOGIES==tech_list[i]].sort_values(by="Date")
    sorted_df=pd.concat([sorted_df,d[["Date","TECHNOLOGIES","energy"]]],ignore_index=True)
fig = px.area(sorted_df, x="Date", y="energy", color="TECHNOLOGIES",line_group="TECHNOLOGIES",title=country+" simulations")
consum=consumption[consumption.AREAS==country]
fig.add_scatter(x=consum.Date, y=consum["total_consumption"], name="Consumption",mode="lines",line_color="black")
fig.show()

## c. HydroStorage operations

In [ ]:
stocklevels=[Variables["stockLevel"]]
for area in stocklevels[0].AREAS.unique():
    fig, ax = plt.subplots(1,figsize=(20,10))
    fig.suptitle(area+" Hydrostorage stock level")
    v=stocklevels[0]
    u=v[v.AREAS==area]
    u=u[u.STOCK_TECHNO=="HydroStorage"]
    ax.plot(u.Date,u.stockLevel,"o",color=colors[0],label=states[0])
    ax.legend()
    ax.set_ylabel("Storage (MWh)")
    plt.xlabel("Date")

## d. Exchanges

In [ ]:

c1,c2="FR","CH"

plt.figure(figsize=(10,5))

fig=plt.figure(figsize=(20,5))
plt.title(c1+'-'+c2+" exchanges")
    
u=exch_list[0]
v=u[(u.AREAS==c1)&(u.AREAS1==c2)]
v.TIMESTAMP=pd.to_datetime(v.TIMESTAMP)
plt.scatter(v.TIMESTAMP,v.exchange,label=states[0],s=20,alpha=0.3,color=colors[0])
plt.legend()
plt.ylabel('Exchanges (MW)')
# plt.subplots_adjust(left=None, bottom=0, right=None, top=1, wspace=0.2, hspace=None)
plt.xlabel("Date")

In [ ]:
ext_fr=["DE","BE","GB","IT","CH","ES"]
exchanges_fr=pd.DataFrame(columns=["state",'countries',"Exchanges (TWh)"])
other_exchanges=pd.DataFrame(columns=["state",'countries',"Exchanges (TWh)"])

for i in range(len(exch_list)):
    s=0
    exchange=exch_list[i]
    for c in ["DE","BE","GB","IT","CH","ES"]:
        value=round(exchange[(exchange.AREAS=="FR") & (exchange.AREAS1==c)].exchange.sum()/10**6,2)
        s+=value
        u=pd.DataFrame(np.array([[states[i],c,
                                 value]]),
                       columns=["state",'countries',"Exchanges (TWh)"])
        exchanges_fr=pd.concat([exchanges_fr,u],ignore_index=True)

    u=pd.DataFrame(np.array([[states[i],"Total",
                                 s]]),
                       columns=["state",'countries',"Exchanges (TWh)"])
    exchanges_fr=pd.concat([exchanges_fr,u],ignore_index=True)

    for a,b in [["CH","DE"],["CH","IT"],["DE","IT"]]:
        value=round(exchange[(exchange.AREAS==a) & (exchange.AREAS1==b)].exchange.sum()/10**6,2)
        u=pd.DataFrame(np.array([[states[i],a+"-"+b,
                                 value]]),
                       columns=["state",'countries',"Exchanges (TWh)"])
        other_exchanges=pd.concat([other_exchanges,u],ignore_index=True)
        
exchanges_fr["Exchanges (TWh)"]=exchanges_fr["Exchanges (TWh)"].astype(float)
other_exchanges["Exchanges (TWh)"]=other_exchanges["Exchanges (TWh)"].astype(float)

In [ ]:
ax = sns.barplot(x="countries", y="Exchanges (TWh)", hue="state",data=exchanges_fr)
ax.title.set_text("France exchanges")

In [ ]:
ax = sns.barplot(x="countries", y="Exchanges (TWh)", hue="state",data=other_exchanges)
ax.title.set_text("Other exchanges")

## e. Cost 

In [ ]:
capacityCost=np.array([Variables['capacityCosts'].capacityCosts.sum()/10**9])

In [ ]:
energyCost=np.array([Variables['energyCosts'].energyCosts.sum()/10**9])

In [ ]:
ax=sns.barplot(x=["Scenario"],y=capacityCost)
plt.ylabel("B€/year")
plt.title("Annualized capacity cost")
ax.bar_label(ax.containers[0])

In [ ]:
ax=sns.barplot(x=["Scenario"],y=energyCost)
plt.ylabel("B€/year")
plt.title("Annualized energy cost")
ax.bar_label(ax.containers[0])

In [ ]:
ax=sns.barplot(x=["Scenario"],y=energyCost+capacityCost)
plt.ylabel("B€/year")
plt.title("Annualized total cost")
ax.bar_label(ax.containers[0])

In [ ]:
tot_cost=energyCost+capacityCost
ax=sns.barplot(x=["Scenario"],y=tot_cost*10**9/np.array([total_prod])*10**-6)
plt.ylabel("€/MWh")
plt.title("Cost of electricity")
ax.bar_label(ax.containers[0])